In [1]:
import os
import scipy
import numpy as np
import trimesh
from trimesh import viewer
from trimesh.transformations import translation_matrix, rotation_matrix, concatenate_matrices
from scipy.spatial.transform import Rotation
from scipy.spatial import cKDTree

np.set_printoptions(precision=3, suppress=True, linewidth=np.inf)


In [2]:
mesh1 = trimesh.exchange.load.load('wolf1.ply')
mesh2 = mesh1.copy()

# apply an arbitrary transformation to mesh1
mesh1 = mesh1.slice_plane(plane_normal=[0, -1, 0], plane_origin=[10, 10, 10])
T = translation_matrix([0, 0, 0])
xaxis, yaxis, zaxis = [1, 0, 0], [0, 1, 0], [0, 0, 1]
Rx = rotation_matrix(0, xaxis)
Ry = rotation_matrix(np.pi/8, yaxis)
Rz = rotation_matrix(0, zaxis)
M = concatenate_matrices(T, Rx, Ry, Rz)
mesh1.apply_transform(M)

# write scene to html
if os.path.exists("scene.html"):
    os.remove("scene.html")
html = viewer.notebook.scene_to_html((trimesh.util.concatenate([mesh1, mesh2])).scene())
with open("scene.html", "w") as file:
    file.write(html)


In [3]:
def register(source, target, w1=0.1, w2=1.0, max_iter=100):
    # resulting transformation is (R, t)
    transform = (np.eye(3), np.zeros(3))
    x = source.vertices
    y = target.vertices
    N = len(x)
    z = x.copy()
    tree = cKDTree(y)
    for iter in range(max_iter):
        # applies transformation to original x
        x_t = x.dot(transform[0]) + transform[1]
        # build a matrix equivalent to cross product
        X_t = np.stack([
            np.stack((np.zeros(len(x_t)), -x_t[:, 2], x_t[:, 1]), axis=1),
            np.stack((x_t[:, 2], np.zeros(len(x_t)), -x_t[:, 0]), axis=1),
            np.stack((-x_t[:, 1], x_t[:, 0], np.zeros(len(x_t))), axis=1)
        ], axis=1)
        # build blocks of sparse coefficient matrix A
        A_rr = np.einsum('abc,adc->bd', X_t, X_t)
        A_tr = -np.sum(X_t, axis=0)
        A_zir = np.concatenate([X_ti for X_ti in X_t], axis=1)
        A_rt = A_tr.T
        A_tt = N * np.eye(3)
        A_zit = -np.tile(np.eye(3), len(x_t))
        A_rzi = A_zir.T
        A_tzi = A_zit.T
        A_zizj = scipy.sparse.eye(3 * len(x_t)) * (w1 / w2 + 1)
        A_rr = scipy.sparse.csr_matrix(A_rr)
        A_tr = scipy.sparse.csr_matrix(A_tr)
        A_zir = scipy.sparse.csr_matrix(A_zir)
        A_rt = scipy.sparse.csr_matrix(A_rt)
        A_tt = scipy.sparse.csr_matrix(A_tt)
        A_zit = scipy.sparse.csr_matrix(A_zit)
        A_rzi = scipy.sparse.csr_matrix(A_rzi)
        A_tzi = scipy.sparse.csr_matrix(A_tzi)
        A_zizj = scipy.sparse.csr_matrix(A_zizj)
        # assemble coefficient matrix A
        A = scipy.sparse.bmat([[A_rr, A_tr, A_zir],
                               [A_rt, A_tt, A_zit],
                               [A_rzi, A_tzi, A_zizj]], format='csr')
        # build blocks of result vector b
        b_r = np.zeros(3)
        b_t = -np.sum(x_t, axis=0)
        _, i = tree.query(z, k=1)
        b_zi = (w1 / w2 * y[i] + x_t).flatten()
        # assemble result vector b
        b = np.concatenate([b_r, b_t, b_zi])
        # solve system of linear equations Ax = b using conjugate gradient method
        solution, info = scipy.sparse.linalg.cg(A, b, tol=1e-10, maxiter=1000)
        # ensure convergence
        assert info == 0
        # update resulting transformation
        r = solution[:3]
        t = solution[3:6]
        z = solution[6:].reshape(-1, 3)
        R = Rotation.from_euler('xyz', r).as_matrix()
        # ensure that R is a rotation matrix
        assert np.isclose(np.linalg.det(R), 1.0)
        transform = (np.dot(transform[0], R), transform[1] + t)
        print(iter, '\n', transform, '\n')
    return transform, z


In [4]:
(R, t), z = register(mesh1, mesh2, max_iter=100)
print('ground truth\n', M[:3, :3], M[:3, 3])
trans_mesh = trimesh.base.Trimesh(mesh1.vertices.dot(R) + t, mesh1.faces) # z is also interesting

if os.path.exists("scene.html"):
    os.remove("scene.html")
html = viewer.notebook.scene_to_html(trimesh.util.concatenate([trans_mesh, mesh2]).scene())
with open("scene.html", "w") as file:
    file.write(html)


0 
 (array([[ 0.986, -0.078,  0.149],
       [ 0.088,  0.994, -0.065],
       [-0.143,  0.077,  0.987]]), array([-0.182,  1.321, -3.942])) 

1 
 (array([[ 0.977, -0.077,  0.201],
       [ 0.093,  0.993, -0.072],
       [-0.194,  0.089,  0.977]]), array([-0.487,  0.645, -3.24 ])) 

2 
 (array([[ 0.969, -0.033,  0.244],
       [ 0.049,  0.997, -0.063],
       [-0.241,  0.073,  0.968]]), array([-1.562,  0.503, -2.867])) 

3 
 (array([[ 0.96 , -0.   ,  0.279],
       [ 0.016,  0.998, -0.054],
       [-0.279,  0.056,  0.959]]), array([-2.157,  0.494, -2.512])) 

4 
 (array([[ 0.952,  0.02 ,  0.306],
       [-0.005,  0.999, -0.047],
       [-0.306,  0.043,  0.951]]), array([-2.29 ,  0.432, -2.229])) 

5 
 (array([[ 0.945,  0.03 ,  0.325],
       [-0.018,  0.999, -0.041],
       [-0.326,  0.033,  0.945]]), array([-2.233,  0.399, -1.944])) 

6 
 (array([[ 0.94 ,  0.033,  0.339],
       [-0.022,  0.999, -0.036],
       [-0.34 ,  0.026,  0.94 ]]), array([-1.984,  0.396, -1.713])) 

7 
 (array([[